# AI Agent PoC

This notebook tests how an AI agent can perform tasks and augment its knowledge by using a tools construct to call python methods.

## Loading My AI Agent Lib

In [41]:
%load_ext autoreload
%autoreload 2
import sys
import os
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
list_dir = os.listdir(src_path)
sys.path.append(src_path)
from aiagent import AIAgent
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")
agent = AIAgent()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get System Prompt

In [42]:
user_request = "I will now start a two hour ride. At what time will I arrive?"
retrieval_prompt = agent.get_retrieval_prompt(user_request)
print(retrieval_prompt)

# Retrieval Process

## System Prompt 
You are a preparing the data retrieval for a helpful assistant. 
You are getting a user request that the helpful assistant will need to answer.
You are getting a list of tools that you can use to perform tasks and to receive external data to augment your knowledge if needed.
You will analyze the user's request and respond nothing but a JSON object containing:
    * the tool calls and their parameters
    * A description of a plan how you will proceed after you received the results from the tools
Here is the JSON schema for your response:
```json
{'$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'required': ['tool_calls', 'plan'], 'properties': {'tool_calls': {'type': 'array', 'items': {'type': 'object', 'required': ['tool', 'parameters'], 'properties': {'tool': {'type': 'string'}, 'parameters': {'type': 'object', 'additionalProperties': {'type': 'string'}}}}}, 'plan': {'type': 'string'}}}
```    

## User request
I will

In [45]:
#with model.chat_session():
#    response = model.generate(retrieval_prompt, max_tokens=1024)
response = model.generate(retrieval_prompt, max_tokens=1024)
print(response)


## Your response

Here is my JSON response:
```json
{
    "tool_calls": [
        {
            "tool": "time",
            "parameters": {}
        },
        {
            "tool": "search",
            "parameters": {
                "query": "traffic"
            }
        }
    ],
    "plan": "I will use the current time and search results to estimate when I'll arrive."
}
```


In [46]:
processed_response = agent.process_response(response)
print(processed_response)

{'tool_calls': [{'tool': 'time', 'parameters': {}, 'response': '2025-02-26 23:35:11'}, {'tool': 'search', 'parameters': {'query': 'traffic'}, 'response': 'Searching the web for: traffic'}], 'plan': "I will use the current time and search results to estimate when I'll arrive."}


In [48]:
interpretation_prompt = agent.get_interpretation_prompt(user_request, processed_response)
interpretation = model.generate(interpretation_prompt, max_tokens=1024)
print(interpretation)





## Interpretation Process

### Step 1: Understand the user request
The user wants to know at what time they will arrive after a two-hour ride.

### Step 2: Review the retrieval process results
I have received the results of the retrieval process, which include:

* Tool calls:
	+ A call to `time` tool with no parameters and response '2025-02-26 23:35:11' (current time).
	+ A call to `search` tool with query parameter set to `'traffic'` and response is a search result.
* Plan: Use the current time and search results to estimate when I'll arrive.

### Step 3: Analyze the information
To answer the user's question, I need to analyze the information from the retrieval process. The `time` tool call provides me with the current time (`2025-02-26 23:35:11`). The `search` tool call gives me a search result related to traffic.

### Step 4: Make an educated guess
Based on my analysis, I can make an educated guess about when the user will arrive. Since they are starting their ride at this momen